In [2]:
import pandas as pd
import numpy as np
import os
from pathlib import Path

from datetime import datetime, date, time

import gc
import copy

import pyarrow.parquet as pq
import pyarrow as pa

In [3]:
from sklearn.metrics import mean_squared_error, roc_auc_score
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold, KFold

In [4]:
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100

import warnings
warnings.filterwarnings("ignore")

import pytorch_lightning as pl
random_seed=1234
pl.seed_everything(random_seed)

Global seed set to 1234


1234

In [5]:
%%time
# train_file = r'/kaggle/input/amex-agg-data-rev2/agg_train_all_rev2_rev.parquet'
train_file = r'amex\agg_train_all_rev2_rev.parquet'
df=pd.read_parquet(train_file, engine='pyarrow')

Wall time: 17.9 s


In [6]:
corr85_150feats = ['D_48|last', 'R_1|max', 'log_D_44|mean', 'B_9|last', 'log_B_3|last', 'D_61|last', 'D_55|last', 'D_44|last', 'log_B_23|last', 'B_3|last', 'D_48|min', 'B_7|last', 'log_B_11|last', 'D_75|last', 'D_61|mean', 'log_B_22|mean', 'log_B_9|last', 'log_B_11|mean', 'R_1|last', 'log_B_23|max', 'B_4|last', 'B_20|last', 'R_10|max', 'B_19|last', 'B_1|last', 'log_B_4|last', 'R_2|max', 'log_B_40|last', 'log_D_44|min', 'B_9|max', 'log_B_40|mean', 'B_9|min', 'log_B_9|max', 'B_30|nunique', 'log_B_23|min', 'B_16|max', 'R_2|last', 'D_78|max', 'log_B_11|min', 'log_B_9|min', 'R_4|max', 'B_22|last', 'log_B_4|max', 'log_B_3|min', 'D_39|max', 'R_2|mean', 'D_58|min', 'B_7|min', 'D_41|last', 'log_D_41|max', 'S_7|max', 'log_D_41|last', 'D_70|max', 'D_44|min', 'R_4|last', 'R_3|mean', 'D_78|last', 'B_8|min', 'D_55|min', 'D_39|last', 'B_17|mean', 'S_15|max', 'log_B_40|min', 'B_17|last', 'B_38=4.0', 'B_1|min', 'S_7|last', 'R_8|max', 'D_70|last', 'R_15|max', 'B_38|nunique', 'B_3|min', 'D_84|max', 'B_16|min', 'D_43|mean', 'B_38=5.0', 'log_B_26|max', 'D_39|mean', 'P_4|max', 'R_15|mean', 'R_10|last', 'log_D_39|max', 'B_17|min', 'S_3|min', 'D_41|mean', 'B_19|min', 'R_8|mean', 'B_40|min', 'B_20|min', 'R_24|max', 'R_16|mean', 'log_D_39|last', 'R_3|last', 'D_43|last', 'B_28|last', 'log_D_43|mean', 'D_84|last', 'R_16|max', 'D_131|max', 'R_26_mean2std', 'D_81|max', 'log_B_26|last', 'S_15|mean', 'D_72|max', 'R_5|last', 'B_30=2.0', 'B_17_mean2std', 'D_79|max', 'log_B_22|min', 'R_13|max', 'log_B_21|mean', 'R_8|last', 'D_42_mean2std', 'D_130|last', 'R_3|min', 'D_49_mean2std', 'R_24|mean', 'D_120=1.0', 'R_11|mean', 'D_46|mean', 'B_25|last', 'D_53_mean2std', 'log_D_131|last', 'D_59|last', 'R_24|last', 'R_6|max', 'S_15|last', 'R_9_mean2std', 'D_70|min', 'R_20|max', 'D_135_mean2std', 'B_25|min', 'D_89|mean', 'D_132_mean2std', 'R_15|last', 'log_S_5|mean', 'log_D_43|min', 'D_131|min', 'S_20|max', 'D_53|last', 'R_13|mean', 'R_11|max', 'log_D_131|min', 'D_42|max', 'R_17|max', 'B_38=6.0', 'log_B_24|mean', 'B_32|max', 'B_22|min', 'P_4|min']
corr75_150feats = ['D_48|last', 'R_1|max', 'B_9|last', 'log_B_3|last', 'D_44|last', 'log_B_23|last', 'log_B_11|last', 'D_58|last', 'R_1|last', 'log_B_9|mean', 'D_61|min', 'B_4|last', 'R_10|max', 'B_22|max', 'B_19|last', 'B_7|max', 'R_2|max', 'log_D_44|min', 'B_11|last', 'B_9|min', 'B_30|nunique', 'R_2|last', 'log_B_22|last', 'D_39|max', 'R_2|mean', 'log_B_40|max', 'D_41|last', 'B_20|mean', 'log_D_41|max', 'B_23|min', 'S_7|max', 'log_B_4|min', 'D_70|max', 'R_3|mean', 'D_78|last', 'B_8|min', 'D_55|min', 'D_39|last', 'B_17|mean', 'S_15|max', 'log_B_40|min', 'B_38=4.0', 'B_1|min', 'R_8|max', 'R_15|max', 'S_3|last', 'B_38|nunique', 'D_84|max', 'D_43|mean', 'B_38=5.0', 'P_4|max', 'R_10|last', 'S_3|min', 'log_B_26|mean', 'B_40|min', 'R_24|max', 'log_D_39|last', 'B_28|last', 'log_D_43|mean', 'R_16|max', 'log_B_28|max', 'D_131|max', 'R_26_mean2std', 'log_B_26|last', 'S_15|mean', 'D_72|max', 'R_5|last', 'B_30=2.0', 'B_17_mean2std', 'log_B_22|min', 'log_B_21|mean', 'R_3|min', 'D_130|max', 'D_120=1.0', 'R_11|mean', 'D_46|mean', 'B_25|last', 'D_53_mean2std', 'D_59|last', 'R_24|last', 'R_6|max', 'S_15|last', 'R_9_mean2std', 'D_81|mean', 'D_70|min', 'D_135_mean2std', 'D_89|mean', 'R_15|last', 'log_S_5|mean', 'D_131|min', 'D_53|last', 'R_13|mean', 'D_42|max', 'R_17|max', 'B_38=6.0', 'log_B_24|mean', 'B_32|max', 'B_22|min', 'R_20|mean', 'P_4|min', 'S_22|max', 'R_6|last', 'log_D_133|last', 'D_89|last', 'log_D_113|min', 'R_21|last', 'log_D_41|min', 'B_17|max', 'D_133|max', 'D_113|max', 'D_120|nunique', 'log_D_60|min', 'D_114=0.0', 'R_20|last', 'D_133|last', 'B_31|nunique', 'R_25|max', 'R_22|max', 'D_46|last', 'S_22|mean', 'R_1|min', 'B_38=7.0', 'B_24|mean', 'log_B_24|last', 'B_24|last', 'log_D_39|min', 'S_27_mean2std', 'log_D_133|min', 'D_41|min', 'D_140|max', 'B_32|last', 'R_7|max', 'B_21|max', 'R_19|last', 'D_143|last', 'log_D_107|min', 'D_64=U', 'log_B_41|max', 'D_53|min', 'B_14|min', 'R_17|last', 'B_41|last', 'R_16|last', 'D_61|max', 'D_65|max', 'D_107|max', 'S_23|mean', 'D_145|last', 'D_46|min', 'log_B_21|last']
len(corr85_150feats), len(corr75_150feats)

(150, 150)

In [7]:
df[corr85_150feats].isna().sum()[df[corr85_150feats].isna().sum()>0]

Series([], dtype: int64)

In [8]:
df[corr75_150feats].isna().sum()[df[corr75_150feats].isna().sum()>0]

Series([], dtype: int64)

In [9]:
sfr_lists = [
    ['B_9|last', 'P_2|last', 'R_1|mean', 'B_2|mean', 'B_37|last', 'log_B_18|last', 'P_2|mean', 'log_B_3|last', 'R_1|last', 'B_18|last', 'D_48|last', 'B_1|last', 'log_D_44|last', 'B_23|last', 'B_2|last', 'P_2|min', 'D_44|max', 'B_9|max', 'D_48|max', 'B_3|last', 'log_B_9|last', 'log_D_44|mean', 'log_B_3|max', 'R_1|max', 'log_B_9|mean', 'log_B_23|max', 'B_7|last', 'D_62|mean', 'S_25|min', 'B_3|max', 'D_75|last', 'D_61|mean', 'D_62|min', 'log_B_3|mean', 'B_33|mean', 'log_B_11|mean', 'B_18|min', 'D_75|max', 'R_2|last', 'B_6|min', 'D_78|max', 'B_22|last', 'R_2|mean', 'log_B_9|min', 'B_30=1.0', 'B_11|last', 'D_44|mean', 'P_2|max', 'D_44|last', 'log_B_23|last', 'log_D_44|max', 'D_45|mean', 'log_D_41|last', 'D_42|last', 'D_61|last', 'D_42|mean', 'log_B_11|last', 'B_20|mean', 'log_D_39|last', 'B_19|last', 'P_3|last', 'D_41|last', 'D_45|max', 'D_39|last', 'D_39|max', 'D_42|max', 'R_6|last', 'R_3|mean', 'S_7|max', 'S_7|mean', 'log_D_45|max', 'R_4|mean', 'log_B_4|last', 'R_3|max', 'D_42|min', 'log_B_9|max', 'log_B_26|last', 'R_8|mean', 'D_78|last', 'P_3|min', 'B_8|min', 'R_2|max', 'B_6|last', 'D_48|mean', 'S_3|last', 'D_65|last', 'S_7|last', 'B_22|mean', 'D_62|last', 'R_5|max', 'R_7|last', 'S_3|max', 'log_D_45|mean', 'R_3|last', 'D_74|last', 'R_10|max', 'B_8|last', 'D_77|min', 'S_3|mean', 'S_15|max', 'B_26|last', 'B_20|last', 'B_10|last', 'S_23|last', 'B_8|mean', 'P_3|mean', 'R_4|last', 'S_3|min', 'R_10|mean', 'D_43|max', 'D_112|last', 'log_D_39|max', 'R_3|min', 'D_53|last', 'D_52|min', 'D_77|mean', 'D_45|last', 'log_D_45|last', 'D_41|max', 'D_43|last', 'R_16|mean', 'S_23|max', 'D_65|max', 'D_65|mean', 'log_D_43|last', 'B_5|mean', 'R_27|last', 'B_8|max', 'D_47|last', 'S_15|mean', 'D_43|mean', 'B_2|min', 'R_8|max', 'S_23|mean', 'R_27|mean', 'log_D_43|max', 'R_5|mean', 'B_4|last', 'B_22|max', 'D_61|max', 'D_48|min', 'B_38|last', 'D_46|last', 'S_7|min', 'log_B_26|mean', 'log_D_43|mean', 'B_37|max', 'log_B_12|last', 'B_17|min', 'B_20|min'],
    
    ['P_2|last', 'P_2|mean', 'D_44|max', 'B_9|last', 'log_B_18|last', 'P_2|min', 'D_48|last', 'log_B_9|mean', 'B_3|last', 'log_B_9|last', 'B_33|mean', 'R_1|mean', 'B_11|last', 'B_20|last', 'R_1|max', 'R_1|last', 'P_2|max', 'log_B_9|max', 'B_3|mean', 'D_48|max', 'B_6|last', 'log_D_44|mean', 'D_48|mean', 'D_61|last', 'B_18|last', 'log_D_44|max', 'B_2|mean', 'B_20|mean', 'B_7|mean', 'D_44|last', 'B_2|last', 'S_23|mean', 'B_7|last', 'B_37|mean', 'B_23|last', 'log_B_11|last', 'S_23|max', 'D_61|min', 'D_62|max', 'B_1|last', 'D_61|mean', 'log_D_44|last', 'S_23|last', 'log_B_3|last', 'log_D_41|last', 'B_9|max', 'B_30=1.0', 'S_3|last', 'B_4|last', 'B_8|min', 'D_44|mean', 'D_42|mean', 'R_10|mean', 'log_B_12|max', 'D_78|max', 'log_B_11|min', 'log_B_40|last', 'D_56|min', 'log_B_11|max', 'S_3|min', 'R_6|max', 'D_53|last', 'D_39|last', 'D_45|mean', 'D_89|max', 'log_D_39|last', 'log_B_4|max', 'D_39|max', 'D_68=1.0', 'S_3|max', 'log_D_43|last', 'S_3|mean', 'R_3|last', 'log_D_102|max', 'D_84|last', 'D_114|nunique', 'D_109|mean', 'P_4|mean', 'S_25|min', 'B_22|last', 'D_129|max', 'D_77|mean', 'R_19|min', 'B_20|max', 'log_D_125|min', 'R_18|max', 'P_3|min', 'log_D_51|min', 'B_4_mean2std', 'S_7|mean', 'D_52_mean2std', 'D_41|last', 'R_11|mean', 'R_4|min', 'log_B_23|min', 'D_91|last', 'D_42|max', 'R_2|min', 'B_33|min', 'log_D_45|mean'],
    ['P_2|last', 'P_2|mean', 'D_44|max', 'B_9|last', 'log_B_18|last', 'P_2|min', 'B_3|last', 'D_48|last', 'R_1|mean', 'D_44|last', 'B_33|mean', 'P_2|max', 'R_1|max', 'B_11|last', 'B_18|last', 'log_B_9|max', 'log_D_44|max', 'B_7|last', 'log_B_9|last', 'D_48|mean', 'log_D_44|mean', 'B_20|last', 'B_3|mean', 'D_48|max', 'B_23|last', 'log_B_9|mean', 'B_2|mean', 'D_61|last', 'B_6|last', 'R_1|last', 'D_41|last', 'B_7|mean', 'B_20|mean', 'B_37|mean', 'D_61|mean', 'S_23|max', 'D_62|max', 'B_1|last', 'log_D_44|last', 'S_23|mean', 'D_61|min', 'B_30=1.0', 'B_2|last', 'B_19|last', 'S_23|last', 'P_3|mean', 'D_39|last', 'S_3|mean', 'R_10|max', 'D_42|mean', 'B_37|last', 'B_33|last', 'R_7|max', 'B_2|min', 'R_5|last', 'log_D_39|last', 'R_2|last', 'P_3|last', 'R_2|mean', 'S_3|max', 'B_9|max', 'B_23|mean', 'S_7|max', 'D_42|last', 'log_B_11|last', 'D_62|min', 'log_B_11|mean', 'D_45|max', 'log_B_22|last', 'R_4|mean', 'D_45|mean', 'R_2|max', 'log_B_23|last', 'B_4|last', 'B_3|max', 'P_3|min', 'D_58|last', 'D_39|max', 'D_46|last', 'R_3|max', 'log_D_41|last', 'D_77|mean', 'D_112|last', 'D_65|last', 'log_B_3|last', 'D_42|min', 'D_43|last', 'log_B_4|last', 'D_75|last', 'D_43|mean', 'R_6|max', 'R_5|max', 'D_46|mean', 'R_7|mean', 'D_51|mean', 'log_D_45|min', 'R_4|last', 'D_45|last', 'R_6|last', 'log_D_43|last'],
    ['P_2|last', 'B_9|last', 'P_2|mean', 'D_44|max', 'log_B_18|last', 'P_2|min', 'D_48|last', 'B_3|last', 'log_B_9|mean', 'R_1|mean', 'B_33|mean', 'P_2|max', 'B_11|last', 'log_B_9|max', 'B_18|last', 'log_B_9|last', 'R_1|max', 'D_48|mean', 'D_48|max', 'log_D_44|mean', 'log_D_44|max', 'D_61|last', 'B_3|mean', 'B_3|max', 'B_23|last', 'R_10|mean', 'B_20|last', 'B_2|mean', 'B_7|mean', 'R_1|last', 'B_6|last', 'D_44|last', 'log_D_44|last', 'B_1|last', 'D_61|mean', 'S_23|mean', 'B_20|mean', 'S_23|max', 'R_2|last', 'B_30=1.0', 'B_2|last', 'D_61|min', 'D_42|mean', 'D_62|max', 'log_B_11|min', 'R_2|max', 'R_4|mean', 'B_37|last', 'D_62|min', 'log_D_45|max', 'log_B_11|last', 'log_D_39|last', 'B_7|last', 'P_3|mean', 'D_45|max', 'log_B_23|last', 'R_2|mean', 'D_39|last', 'B_10|last', 'D_75|last', 'B_9|max', 'R_5|last', 'B_19|last', 'R_3|mean', 'P_3|min', 'D_42|max', 'log_B_4|last', 'B_2|min', 'R_5|mean', 'D_39|max', 'R_5|max', 'S_7|last', 'log_D_41|last', 'R_6|last', 'S_3|max', 'D_42|min', 'S_7|mean', 'D_78|max', 'D_42|last', 'log_B_3|last', 'D_84|mean', 'log_D_39|max', 'S_3|mean', 'log_D_45|min', 'D_41|last', 'D_41|mean', 'B_8|min', 'R_7|last', 'R_10|max', 'log_B_26|last', 'B_26|last', 'R_3|max', 'S_3|min', 'B_4|last', 'D_75|mean', 'log_B_18|min', 'D_45|last', 'D_77|last', 'S_3|last', 'log_D_45|last'],
    ['P_2|mean', 'P_2|min', 'P_2|last', 'log_B_9|last', 'B_9|mean', 'R_1|last', 'D_48|last', 'D_44|max', 'B_9|max', 'R_1|mean', 'D_44|last', 'B_9|last', 'B_6|last', 'log_B_3|last', 'B_23|mean', 'D_44|mean', 'B_33|last', 'log_B_18|last', 'R_10|max', 'B_2|last', 'B_2|mean', 'B_2|min', 'B_1|last', 'R_1|max', 'R_2|max', 'log_B_9|max', 'R_10|mean', 'log_B_9|mean', 'log_D_44|mean', 'log_B_11|last', 'B_1|mean', 'D_61|mean', 'log_D_44|last', 'B_3|last', 'D_45|max', 'log_D_44|max', 'log_B_11|mean', 'D_42|mean', 'D_42|max', 'S_23|last', 'B_30=1.0', 'D_75|last', 'log_B_3|mean', 'B_17|min', 'B_3|max', 'D_42|last', 'B_26|last', 'D_59|max', 'R_6|last', 'S_6|max', 'B_33|max', 'S_3|mean', 'log_D_39|last', 'D_62|mean', 'R_2|last', 'R_5|last', 'B_11|last', 'D_39|last', 'log_D_39|max', 'B_33|mean', 'D_72|min', 'log_B_22|max', 'B_37|last', 'D_41|last', 'B_18|last', 'D_70_mean2std', 'D_62|min', 'R_3|max', 'R_6|mean', 'D_134|min', 'log_D_43|mean', 'B_23|last', 'R_12|max', 'D_69|min', 'log_B_18|min', 'D_65|last', 'P_3|last', 'S_22|max', 'D_70|last', 'D_130|max', 'S_3|max', 'S_26|last', 'S_15|mean', 'R_27|mean', 'D_133|min', 'S_23|max', 'B_10|last', 'B_5|last', 'D_132|min', 'S_6|min', 'R_3|last', 'R_12|mean', 'log_B_18|mean', 'P_2|max', 'D_61|last', 'log_B_4|last', 'D_74|mean', 'B_25|last', 'log_B_18|max', 'R_13|last'],
    ['P_2|min', 'P_2|mean', 'P_2|last', 'log_B_9|last', 'D_48|last', 'D_44|max', 'log_B_9|max', 'B_9|max', 'D_44|last', 'R_1|mean', 'B_3|last', 'R_10|max', 'B_2|min', 'R_1|max', 'B_6|last', 'log_B_18|last', 'D_44|mean', 'B_2|last', 'B_23|mean', 'log_D_44|max', 'B_33|last', 'B_1|last', 'log_B_3|last', 'log_B_3|mean', 'B_9|last', 'R_1|last', 'D_61|mean', 'B_1|mean', 'B_2|mean', 'D_75|last', 'log_B_9|mean', 'R_10|mean', 'B_17|min', 'log_D_44|mean', 'log_D_44|last', 'B_30=1.0', 'log_B_11|last', 'B_33|mean', 'D_42|last', 'S_3|mean', 'log_B_11|mean', 'B_33|max', 'log_B_4|last', 'log_D_131|min', 'D_39|max', 'log_B_22|max', 'log_D_39|last', 'D_42|mean', 'D_78|last', 'D_62|min', 'D_122|min', 'D_42|max', 'B_9|mean', 'S_9|max', 'D_45|max', 'B_26|last', 'B_37|last', 'log_D_41|last', 'D_128|min', 'log_B_26|last', 'D_43|last', 'log_B_40|last', 'log_D_49|last', 'D_112|last', 'D_62|mean', 'D_46_mean2std', 'S_7|mean', 'D_122|last', 'D_53|last', 'S_7|last', 'D_46|last', 'R_6|mean', 'B_5|last', 'D_139|mean', 'log_D_45|min', 'log_D_131|last', 'log_B_4|mean', 'B_40|last', 'R_4|max', 'R_6|last', 'D_41|last', 'D_43|max', 'log_D_51|max', 'P_3|last', 'D_105|max', 'D_39|last', 'S_25|mean', 'log_D_118|mean', 'B_33|min', 'log_D_43|max', 'D_106|last', 'P_2|max', 'R_10|last', 'D_81|mean', 'B_3|min', 'S_25|last', 'D_129|last', 'log_D_123|last', 'D_123|mean', 'D_46|min'],
    ['P_2|mean', 'P_2|min', 'P_2|last', 'log_B_9|last', 'D_48|last', 'R_10|max', 'D_44|max', 'log_B_9|max', 'R_1|mean', 'B_9|max', 'log_D_44|max', 'D_44|last', 'B_3|last', 'B_6|last', 'log_B_3|last', 'B_2|last', 'R_1|max', 'B_23|mean', 'D_44|mean', 'B_1|last', 'R_1|last', 'log_B_18|last', 'R_10|mean', 'B_2|mean', 'B_33|last', 'log_D_44|mean', 'B_2|min', 'log_B_9|mean', 'B_9|last', 'log_B_3|mean', 'B_33|mean', 'D_75|last', 'D_61|mean', 'P_2|max', 'B_1|mean', 'B_9|mean', 'log_B_11|mean', 'log_B_22|max', 'log_B_11|last', 'B_30=1.0', 'log_D_44|last', 'R_2|last', 'B_17|min', 'B_33|max', 'D_42|max', 'D_42|last', 'D_61|last', 'R_2|max', 'P_3|last', 'log_D_41|last', 'B_37|last', 'B_18|last', 'P_3|mean', 'D_41|last', 'D_42|min', 'log_B_18|min', 'B_26|last', 'R_8|mean', 'log_D_39|last', 'S_3|mean', 'R_3|max', 'log_B_4|last', 'log_D_45|max', 'D_112|last', 'B_11|last', 'B_23|last', 'B_38|last', 'D_42|mean', 'B_4|last', 'R_4|max', 'D_39|last', 'R_5|last', 'R_3|mean', 'D_46|last', 'S_23|last', 'D_62|mean', 'S_25|last', 'D_45|max', 'B_19|last', 'S_3|max', 'S_3|min', 'S_7|last', 'R_2|mean', 'R_5|max', 'R_4|mean', 'R_5|mean', 'D_39|max', 'log_B_23|last', 'log_D_45|last', 'R_7|last', 'log_D_39|max', 'log_D_45|mean', 'B_25|last', 'S_7|max', 'R_7|max', 'B_22|last', 'R_6|mean', 'D_45|mean', 'D_62|last', 'P_4|mean'],
    ['B_9|mean', 'log_B_9|last', 'B_20|last', 'R_1|last', 'log_B_11|last', 'D_48|max', 'P_2|min', 'P_2|last', 'log_D_44|last', 'P_2|max', 'R_1|mean', 'log_B_9|mean', 'P_2|mean', 'D_48|last', 'D_61|last', 'log_B_23|last', 'B_18|last', 'R_10|mean', 'D_44|max', 'B_9|max', 'D_75|last', 'B_11|last', 'B_2|min', 'D_44|mean', 'B_23|last', 'log_D_44|max', 'log_B_18|last', 'log_B_11|mean', 'D_44|last', 'B_7|last', 'R_1|max', 'B_1|last', 'B_11|mean', 'D_78|last', 'B_10|mean', 'B_37|last', 'B_4|last', 'log_D_39|max', 'D_39|last', 'B_9|last', 'log_D_45|max', 'B_1|max', 'log_B_3|last', 'D_42|max', 'S_25|min', 'log_B_3|mean', 'R_5|mean', 'D_42|last', 'R_2|last', 'log_D_44|mean', 'D_42|min', 'D_41|last', 'log_D_39|last', 'D_42|mean', 'log_D_41|last', 'log_B_4|last', 'R_4|mean', 'R_2|max', 'S_3|last', 'log_D_45|min', 'B_8|min', 'R_3|mean', 'D_65|max', 'D_46|last', 'D_52|min', 'R_27|min', 'S_23|last', 'B_15|mean', 'D_129|max', 'P_3|last', 'D_78|mean', 'B_33|max', 'R_8|max', 'B_2|last', 'D_43|last', 'P_3|min', 'B_11|min', 'B_19|last', 'log_B_40|last', 'S_23|max', 'log_D_43|last', 'R_3|max', 'D_112|last', 'R_16|mean', 'log_B_22|last', 'R_10|max', 'log_B_22|mean', 'D_75|max', 'B_3|last', 'R_10|last', 'log_D_107|mean', 'D_39|max', 'R_2|mean', 'S_22|last', 'R_13_mean2std', 'S_7|last', 'R_6|mean', 'D_65|last', 'log_B_18|min', 'D_102|max'],
    ['B_9|mean', 'B_20|last', 'log_B_9|last', 'R_1|last', 'D_48|max', 'log_B_11|last', 'P_2|mean', 'P_2|last', 'P_2|min', 'P_2|max', 'D_61|last', 'log_B_23|last', 'log_D_44|last', 'B_18|last', 'R_10|mean', 'D_48|last', 'R_1|mean', 'log_B_3|last', 'B_9|last', 'D_75|last', 'B_9|max', 'B_11|last', 'D_44|max', 'log_B_9|mean', 'B_2|min', 'log_B_18|last', 'D_44|mean', 'B_23|last', 'log_B_11|mean', 'log_D_44|max', 'B_7|last', 'B_4|last', 'D_42|last', 'D_44|last', 'B_37|last', 'B_10|mean', 'log_D_39|last', 'B_2|last', 'D_78|last', 'B_11|mean', 'log_D_44|mean', 'R_5|mean', 'P_3|mean', 'log_B_3|mean', 'S_25|min', 'R_2|last', 'R_7|max', 'R_1|max', 'B_1|last', 'S_7|mean', 'P_3|last', 'log_D_45|last', 'B_1|max', 'B_33|last', 'S_3|last', 'D_42|mean', 'R_2|mean', 'B_33|max', 'log_D_41|max', 'R_6|mean', 'B_3|last', 'D_45|min', 'B_22|last', 'D_77|mean', 'D_45|max', 'log_B_40|last', 'log_D_45|min', 'B_4|max', 'log_B_26|last', 'D_42|min', 'D_46|last', 'D_48|mean', 'log_D_39|max', 'S_3|mean', 'D_39|last', 'D_78|mean', 'B_3|mean', 'D_42|max', 'log_B_18|mean', 'D_39|max', 'R_10|max', 'B_33|min', 'D_41|max', 'D_58|last', 'D_52|mean', 'D_61|mean', 'R_6|max', 'B_26|last', 'D_60|max', 'S_8|last', 'log_D_43|max', 'S_3|max', 'D_45|last', 'S_17|min', 'B_37|mean', 'S_25|last', 'D_122|max', 'B_21|max', 'log_B_4|last', 'B_25|min'],
    ['B_9|mean', 'log_B_9|last', 'B_20|last', 'R_1|last', 'log_B_11|last', 'D_48|max', 'P_2|min', 'P_2|last', 'R_1|mean', 'P_2|mean', 'B_18|last', 'log_D_44|max', 'D_61|last', 'D_48|last', 'log_B_23|last', 'R_10|mean', 'log_D_44|last', 'log_B_3|last', 'D_44|mean', 'P_2|max', 'D_44|max', 'B_9|max', 'R_1|max', 'B_23|last', 'log_B_9|mean', 'log_B_18|last', 'B_2|last', 'B_9|last', 'B_2|min', 'D_75|last', 'B_37|last', 'log_B_11|mean', 'B_4|last', 'B_7|last', 'B_1|last', 'B_11|mean', 'D_44|last', 'log_D_44|mean', 'B_1|max', 'log_B_3|mean', 'D_61|mean', 'B_10|mean', 'R_2|last', 'R_2|mean', 'R_2|max', 'D_42|min', 'D_78|last', 'D_48|mean', 'S_25|min', 'B_3|last', 'D_78|max', 'log_D_41|last', 'D_61|max', 'log_D_45|last', 'S_7|mean', 'log_D_39|last', 'log_D_45|min', 'D_42|max', 'P_3|min', 'D_39|last', 'R_6|mean', 'R_3|mean', 'D_42|last', 'log_B_9|max', 'P_3|last', 'B_8|min', 'S_3|last', 'D_62|min', 'D_45|mean', 'R_5|max', 'B_30=0.0', 'D_42|mean', 'B_11|last', 'R_27|max', 'D_41|max', 'S_15|mean', 'B_33|max', 'P_3|mean', 'B_1|mean', 'R_3|max', 'R_6|last', 'log_B_4|last', 'R_5|last', 'D_39|max', 'D_52|max', 'log_D_45|mean', 'log_B_26|last', 'log_D_39|max', 'log_D_43|last', 'R_27|min', 'D_46|last', 'D_128|max', 'D_128|min', 'S_3|mean', 'log_D_51|max', 'D_78|mean', 'R_7|mean', 'R_27|last', 'D_62|last', 'S_8|min'],    
    
]

union_list = set()
inter_list = set(sfr_lists[0])
for sfr_ in sfr_lists:
    union_list = union_list | set(sfr_)
    inter_list = inter_list & set(sfr_)

union_list = list(union_list)
inter_list = list(inter_list)
union_list.sort()
inter_list.sort()
print(len(union_list), union_list)
print(len(inter_list), inter_list)

276 ['B_10|last', 'B_10|mean', 'B_11|last', 'B_11|mean', 'B_11|min', 'B_15|mean', 'B_17|min', 'B_18|last', 'B_18|min', 'B_19|last', 'B_1|last', 'B_1|max', 'B_1|mean', 'B_20|last', 'B_20|max', 'B_20|mean', 'B_20|min', 'B_21|max', 'B_22|last', 'B_22|max', 'B_22|mean', 'B_23|last', 'B_23|mean', 'B_25|last', 'B_25|min', 'B_26|last', 'B_2|last', 'B_2|mean', 'B_2|min', 'B_30=0.0', 'B_30=1.0', 'B_33|last', 'B_33|max', 'B_33|mean', 'B_33|min', 'B_37|last', 'B_37|max', 'B_37|mean', 'B_38|last', 'B_3|last', 'B_3|max', 'B_3|mean', 'B_3|min', 'B_40|last', 'B_4_mean2std', 'B_4|last', 'B_4|max', 'B_5|last', 'B_5|mean', 'B_6|last', 'B_6|min', 'B_7|last', 'B_7|mean', 'B_8|last', 'B_8|max', 'B_8|mean', 'B_8|min', 'B_9|last', 'B_9|max', 'B_9|mean', 'D_102|max', 'D_105|max', 'D_106|last', 'D_109|mean', 'D_112|last', 'D_114|nunique', 'D_122|last', 'D_122|max', 'D_122|min', 'D_123|mean', 'D_128|max', 'D_128|min', 'D_129|last', 'D_129|max', 'D_130|max', 'D_132|min', 'D_133|min', 'D_134|min', 'D_139|mean', '

In [10]:
x_cols = list(set(union_list + corr85_150feats + corr75_150feats))
len(x_cols)

393

In [11]:
x_cols = ['B_10|last', 'B_10|mean', 'B_11|last', 'B_11|mean', 'B_11|min', 'B_14|last', 'B_14|min', 'B_15|mean', 'B_16|max', 'B_16|min', 'B_17_mean2std', 'B_17|last', 'B_17|max', 'B_17|mean', 'B_17|min', 'B_18|last', 'B_18|min', 'B_19|last', 'B_19|min', 'B_1|last', 'B_1|max', 'B_1|mean', 'B_1|min', 'B_20|last', 'B_20|max', 'B_20|mean', 'B_20|min', 'B_21|last', 'B_21|max', 'B_22|last', 'B_22|max', 'B_22|mean', 'B_22|min', 'B_23|last', 'B_23|mean', 'B_23|min', 'B_24|last', 'B_24|mean', 'B_25|last', 'B_25|min', 'B_26|last', 'B_28|last', 'B_2|last', 'B_2|mean', 'B_2|min', 'B_30=0.0', 'B_30=1.0', 'B_30=2.0', 'B_30|nunique', 'B_31|nunique', 'B_32|last', 'B_32|max', 'B_33|last', 'B_33|max', 'B_33|mean', 'B_33|min', 'B_37|last', 'B_37|max', 'B_37|mean', 'B_38=4.0', 'B_38=5.0', 'B_38=6.0', 'B_38=7.0', 'B_38|last', 'B_38|nunique', 'B_3|last', 'B_3|max', 'B_3|mean', 'B_3|min', 'B_40|last', 'B_40|min', 'B_41|last', 'B_4_mean2std', 'B_4|last', 'B_4|max', 'B_5|last', 'B_5|mean', 'B_6|last', 'B_6|min', 'B_7|last', 'B_7|max', 'B_7|mean', 'B_7|min', 'B_8|last', 'B_8|max', 'B_8|mean', 'B_8|min', 'B_9|last', 'B_9|max', 'B_9|mean', 'B_9|min', 'D_102|max', 'D_105|max', 'D_106|last', 'D_107|max', 'D_109|mean', 'D_112|last', 'D_113|max', 'D_114=0.0', 'D_114|nunique', 'D_120=1.0', 'D_120|nunique', 'D_122|last', 'D_122|max', 'D_122|min', 'D_123|mean', 'D_128|max', 'D_128|min', 'D_129|last', 'D_129|max', 'D_129|mean', 'D_130|last', 'D_130|max', 'D_131|max', 'D_131|min', 'D_132_mean2std', 'D_132|min', 'D_133|last', 'D_133|max', 'D_133|min', 'D_134|min', 'D_135_mean2std', 'D_139|mean', 'D_140|max', 'D_143|last', 'D_145|last', 'D_39|last', 'D_39|max', 'D_39|mean', 'D_41|last', 'D_41|max', 'D_41|mean', 'D_41|min', 'D_42_mean2std', 'D_42|last', 'D_42|max', 'D_42|mean', 'D_42|min', 'D_43|last', 'D_43|max', 'D_43|mean', 'D_43|min', 'D_44|last', 'D_44|max', 'D_44|mean', 'D_44|min', 'D_45|last', 'D_45|max', 'D_45|mean', 'D_45|min', 'D_46_mean2std', 'D_46|last', 'D_46|mean', 'D_46|min', 'D_47|last', 'D_48|last', 'D_48|max', 'D_48|mean', 'D_48|min', 'D_49_mean2std', 'D_51|last', 'D_51|mean', 'D_52_mean2std', 'D_52|last', 'D_52|max', 'D_52|mean', 'D_52|min', 'D_53_mean2std', 'D_53|last', 'D_53|max', 'D_53|min', 'D_54|last', 'D_55|last', 'D_55|min', 'D_56|min', 'D_58|last', 'D_58|min', 'D_59|last', 'D_59|max', 'D_60|max', 'D_61|last', 'D_61|max', 'D_61|mean', 'D_61|min', 'D_62|last', 'D_62|max', 'D_62|mean', 'D_62|min', 'D_64=U', 'D_65|last', 'D_65|max', 'D_65|mean', 'D_68=1.0', 'D_69|min', 'D_70_mean2std', 'D_70|last', 'D_70|max', 'D_70|min', 'D_72|max', 'D_72|min', 'D_74|last', 'D_74|max', 'D_74|mean', 'D_75|last', 'D_75|max', 'D_75|mean', 'D_77|last', 'D_77|max', 'D_77|mean', 'D_77|min', 'D_78|last', 'D_78|max', 'D_78|mean', 'D_79|max', 'D_81|max', 'D_81|mean', 'D_84|last', 'D_84|max', 'D_84|mean', 'D_89|last', 'D_89|max', 'D_89|mean', 'D_91|last', 'P_2|last', 'P_2|max', 'P_2|mean', 'P_2|min', 'P_3|last', 'P_3|max', 'P_3|mean', 'P_3|min', 'P_4|max', 'P_4|mean', 'P_4|min', 'R_10|last', 'R_10|max', 'R_10|mean', 'R_11|last', 'R_11|max', 'R_11|mean', 'R_12|max', 'R_12|mean', 'R_13_mean2std', 'R_13|last', 'R_13|max', 'R_13|mean', 'R_15|last', 'R_15|max', 'R_15|mean', 'R_16|last', 'R_16|max', 'R_16|mean', 'R_17|last', 'R_17|max', 'R_18|max', 'R_19|last', 'R_19|min', 'R_1|last', 'R_1|max', 'R_1|mean', 'R_1|min', 'R_20|last', 'R_20|max', 'R_20|mean', 'R_21|last', 'R_22|max', 'R_24|last', 'R_24|max', 'R_24|mean', 'R_25|max', 'R_26_mean2std', 'R_27|last', 'R_27|max', 'R_27|mean', 'R_27|min', 'R_2|last', 'R_2|max', 'R_2|mean', 'R_2|min', 'R_3|last', 'R_3|max', 'R_3|mean', 'R_3|min', 'R_4|last', 'R_4|max', 'R_4|mean', 'R_4|min', 'R_5|last', 'R_5|max', 'R_5|mean', 'R_6|last', 'R_6|max', 'R_6|mean', 'R_7|last', 'R_7|max', 'R_7|mean', 'R_8|last', 'R_8|max', 'R_8|mean', 'R_9_mean2std', 'S_11|mean', 'S_12|max', 'S_15|last', 'S_15|max', 'S_15|mean', 'S_17|min', 'S_20|max', 'S_22|last', 'S_22|max', 'S_22|mean', 'S_23|last', 'S_23|max', 'S_23|mean', 'S_25|last', 'S_25|mean', 'S_25|min', 'S_26|last', 'S_27_mean2std', 'S_3|last', 'S_3|max', 'S_3|mean', 'S_3|min', 'S_6|max', 'S_6|min', 'S_7|last', 'S_7|max', 'S_7|mean', 'S_7|min', 'S_8|last', 'S_8|mean', 'S_8|min', 'S_9|max', 'log_B_11|last', 'log_B_11|max', 'log_B_11|mean', 'log_B_11|min', 'log_B_12|last', 'log_B_12|max', 'log_B_18|last', 'log_B_18|max', 'log_B_18|mean', 'log_B_18|min', 'log_B_21|last', 'log_B_21|mean', 'log_B_22|last', 'log_B_22|max', 'log_B_22|mean', 'log_B_22|min', 'log_B_23|last', 'log_B_23|max', 'log_B_23|min', 'log_B_24|last', 'log_B_24|mean', 'log_B_26|last', 'log_B_26|max', 'log_B_26|mean', 'log_B_3|last', 'log_B_3|max', 'log_B_3|mean', 'log_B_3|min', 'log_B_40|last', 'log_B_40|max', 'log_B_40|mean', 'log_B_40|min', 'log_B_41|max', 'log_B_4|last', 'log_B_4|max', 'log_B_4|mean', 'log_B_4|min', 'log_B_5|last', 'log_B_9|last', 'log_B_9|max', 'log_B_9|mean', 'log_B_9|min', 'log_D_102|max', 'log_D_107|mean', 'log_D_107|min', 'log_D_113|min', 'log_D_118|mean', 'log_D_123|last', 'log_D_125|min', 'log_D_131|last', 'log_D_131|min', 'log_D_133|last', 'log_D_133|min', 'log_D_39|last', 'log_D_39|max', 'log_D_39|min', 'log_D_41|last', 'log_D_41|max', 'log_D_41|min', 'log_D_43|last', 'log_D_43|max', 'log_D_43|mean', 'log_D_43|min', 'log_D_44|last', 'log_D_44|max', 'log_D_44|mean', 'log_D_44|min', 'log_D_45|last', 'log_D_45|max', 'log_D_45|mean', 'log_D_45|min', 'log_D_49|last', 'log_D_51|max', 'log_D_51|min', 'log_D_60|min', 'log_S_5|mean']
len(x_cols)

408

## hyperopt parameters

In [12]:
learn_rates = np.concatenate((
#                            np.arange(0.00001, 0.0001, 0.00001),  
                           np.arange(0.0001, 0.001, 0.0001), 
                           np.arange(0.001, 0.01, 0.001), 
                           np.arange(0.01, 0.1, 0.01)
                          ), 
                          axis=0)


len(learn_rates)

27

In [13]:
#https://lightgbm.readthedocs.io/en/latest/Parameters.html
#https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMClassifier.html?highlight=classifier#lightgbm.LGBMClassifier

from hyperopt import hp
import numpy as np
space  = { 
    
                 'n_estimators': hp.choice('n_estimators', range(500, 2000, 50)),#num_boost_round
                 'boosting_type':hp.choice('boosting_type', ['gbdt']),#boosting
                 'objective':hp.choice('objective', ['binary'] ),
                 'metric':hp.choice('metric', ['auc']),
                 'learning_rate':  hp.choice('learning_rate', learn_rates), 
                 'colsample_bytree': hp.choice('colsample_bytree', np.round(np.arange(0.1, 0.86, 0.05),3)), #feature_fraction
                 'max_depth': hp.choice('max_depth', range(7, 36, 1)), #int type
                 'min_child_samples': hp.choice('min_child_samples',range(100, 5001, 50)), #min_data_in_leaf
                 'reg_alpha':hp.choice('reg_alpha', [0.0001, 0.001, 0.01, 0.05, 0.1, 0.5, 1, 5, 10,15]),#lambda_l1
                 'reg_lambda':hp.choice('reg_lambda', [0.0001, 0.001, 0.01, 0.05, 0.1, 0.5, 1, 5, 10,15]),#lambda_l2
                 'max_bin':hp.choice('max_bin', range(500, 10000, 50)),
                 'min_data_in_bin':hp.choice('min_data_in_bin', range(500, 9000, 50)),
                 'subsample':hp.choice('subsample', np.round(np.arange(0.1, 0.96, 0.05),3)), #bagging_fraction
                 'subsample_freq':hp.choice('subsample_freq', range(1, 100, 2)),#bagging_freq
                 #max number of leaves in one tree. 1 < num_leaves <= 131072. classes< num_leaves< 2^max_depth  
                 'num_leaves':hp.choice('num_leaves', range(31, 300, 5)),#max_leaves. 
                 'random_state':hp.choice('random_state', [1234]),
                 'n_jobs':hp.choice('n_jobs', [4]),#nthread
                 #'min_split_gain':hp.choice('min_split_gain', [0.0]), #min_gain_to_split
                 #'min_child_weight':hp.choice('min_child_weight', [0.001]),   #min_sum_hessian_in_leaf
                 #'subsample_for_bin':hp.choice('subsample_for_bin', [200000]),   #bin_construct_sample_cnt 
                 #'importance_type':hp.choice('importance_type', ['split']),   
    
                  }
                  
    


### lightgbm

In [14]:
import lightgbm as lgb
def train_trees(X_train, y_train, num_round=100, params={} ):
    
    params['verbosity'] = -1
    dtrain = lgb.Dataset(X_train, y_train)
    
    tree_model = lgb.train(params,
                dtrain,
                num_boost_round=num_round)
    
    
    del dtrain
    gc.collect()

    return tree_model

In [15]:
# @yunchonggan's fast metric implementation
# From https://www.kaggle.com/competitions/amex-default-prediction/discussion/328020
# https://www.kaggle.com/code/ambrosm/amex-lightgbm-quickstart
def amex_metric(y_true: np.array, y_pred: np.array) -> float:

    # count of positives and negatives
    n_pos = y_true.sum()
    n_neg = y_true.shape[0] - n_pos

    # sorting by descring prediction values
    indices = np.argsort(y_pred)[::-1]
    preds, target = y_pred[indices], y_true[indices]

    # filter the top 4% by cumulative row weights
    weight = 20.0 - target * 19.0
    cum_norm_weight = (weight / weight.sum()).cumsum()
    four_pct_filter = cum_norm_weight <= 0.04

    # default rate captured at 4%
    d = target[four_pct_filter].sum() / n_pos

    # weighted gini coefficient
    lorentz = (target / n_pos).cumsum()
    gini = ((lorentz - cum_norm_weight) * weight).sum()

    # max weighted gini coefficient
    gini_max = 10 * n_neg * (1 - 19 / (n_pos + 20 * n_neg))

    # normalized weighted gini coefficient
    g = gini / gini_max

    return 0.5 * (g + d)


In [16]:
X = df[x_cols]
y = df[['target']]

In [17]:
skf = KFold(n_splits=3)
skf.get_n_splits(X, y)

3

In [18]:
print(skf)

for train_index, test_index in skf.split(X, y):
#     print("TRAIN:", train_index, "TEST:", test_index)
#     print("TRAIN:", len(train_index), "TEST:", len(test_index), len(test_index)/(len(test_index)+len(train_index)))
    print(y.iloc[test_index]['target'].value_counts()/len(test_index)) 
    print(y.iloc[train_index]['target'].value_counts()/len(train_index))

KFold(n_splits=3, random_state=None, shuffle=False)
0    0.740565
1    0.259435
Name: target, dtype: float64
0    0.741317
1    0.258683
Name: target, dtype: float64
0    0.739552
1    0.260448
Name: target, dtype: float64
0    0.741824
1    0.258176
Name: target, dtype: float64
0    0.743082
1    0.256918
Name: target, dtype: float64
0    0.740059
1    0.259941
Name: target, dtype: float64


In [19]:
len(train_index), len(test_index)

(305942, 152971)

In [20]:
loss_dict = []


def score(params):
#     print(params)
    num_boost_round = params['n_estimators']
    params_ = copy.deepcopy(params)
    del params_['n_estimators']
    
    losses = []
    
    for train_index, test_index in skf.split(X, y):
        
        #----start: data prep-------------------------------------
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        #----end: data prep-------------------------------------
        
        #-----start: train trees---------------------------------------
        tree_model = train_trees(X_train, y_train['target'].values,
                                 num_round=num_boost_round, 
                                 params=params_
                                )

        y_preds = tree_model.predict(X_test, num_iteration=tree_model.best_iteration)
#         loss = roc_auc_score(y_test, y_preds)
#         loss = f1_score(y_test['target'].values, (y_preds>=0.9).astype(int))
#         loss = amex_metric(y_test, 
#                            pd.DataFrame(data={'prediction': y_preds}))
        loss = amex_metric(y_test['target'].values, y_preds)
        losses.append(loss)
        #-----end: train trees---------------------------------------
        
    loss = np.mean(losses)
#     print(loss)
    loss_dict.append({'params': params, 'losses': losses, 'mean_loss': loss})
    
    if len(loss_dict)%10==0:
        pd.DataFrame(data=loss_dict).to_excel('amex-hyperopt-lgb-rev2-spsfr-3kfold-corr75-union3.xlsx', index=False)
    return {'loss': -loss, 'status': STATUS_OK}

In [21]:
from hyperopt import hp
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, anneal, rand
from functools import partial
def optimize(space, evals, cores, trials, optimizer=tpe.suggest, random_state=1234, n_startup_jobs=10):
    algo = partial(optimizer, n_startup_jobs=n_startup_jobs)
    best = fmin(score, space, algo=algo, max_evals=evals, trials = trials)
    print(best)
    return best

In [24]:
cores = 4
n=500
verbose = False
trials = Trials()

In [ ]:
best_param = optimize(space,
                      evals = n,
                      optimizer=tpe.suggest,
                      cores = cores,
                      trials = trials, random_state=1234, 
                      n_startup_jobs=10)

In [31]:
pd.DataFrame(data=loss_dict).to_excel('amex-hyperopt-lgb-rev2-spsfr-3kfold-corr75-union3.xlsx', index=False)